# Digit Recognizer with CNN

In this notebook, we build and evaluate a Convolutional Neural Network (CNN) to recognize hand-drawn digits from the MNIST-like dataset provided by Kaggle's Digit Recognizer competition. Our objective is to preprocess the training data, augment it, and use a CNN to train a model capable of accurately predicting the digits in the test set. Finally, we format and save the predictions for submission in the required Kaggle format. The steps include data preparation, model training, data augmentation, and evaluation on the validation set before making predictions on the test set.

## First Part - Creating the model

In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense

### **Loading the Training Data**
Here, we load the `train.csv` and `test.csv` files containing the images of digits and their associated labels. The images are stored as pixel values, and the labels represent the drawn digits.


In [30]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### **Preparing the Data**
The images are resized and normalized to make them compatible with the model (values between 0 and 1). We also perform reshaping to ensure the images have the correct shape (`28x28x1`).


In [31]:
train_images = train_df.iloc[:, 1:].values
train_labels = train_df.iloc[:, 0].values

test_images = test_df.values

train_images = train_images.reshape((-1, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((-1, 28, 28, 1)).astype('float32') / 255

### **Splitting the Data into Training and Validation Sets**
We split the training data into two subsets: one for training and one for validation. This allows us to check the model's performance during training on data it has not seen.


In [32]:
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

### **Defining the ImageDataGenerator for Data Augmentation**
We use `ImageDataGenerator` to apply transformations to the images during training. These transformations include rotation, translation, zoom, and more, which helps increase the diversity of the data.

In [33]:
datagen = ImageDataGenerator(
    rotation_range=10,  # Rotation de -10 à +10 degrés
    width_shift_range=0.1,  # Déplacement horizontal
    height_shift_range=0.1,  # Déplacement vertical
    zoom_range=0.1,  # Zoom entre 90% et 110%
    shear_range=0.1,  # Décalage (shear)
    horizontal_flip=False,  # Pas de retournement horizontal
    fill_mode='nearest'  # Compléter les pixels manquants par la valeur la plus proche
)

datagen.fit(train_images)

### Defining the Model

In this enhanced approach, we define a convolutional neural network (CNN) with multiple convolutional layers, each followed by batch normalization and dropout for improved stability and regularization. MaxPooling layers are used to reduce spatial dimensions, and the network concludes with a dense layer of 10 neurons, corresponding to the digits from 0 to 9. L2 regularization is applied to the dense layer to prevent overfitting, and a learning rate scheduler is employed to optimize the training process dynamically.

In [34]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(28, 28, 1), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dense(10, activation='softmax')
])

### **Compiling the Model**
We compile the model using the `Adam` optimizer and the `sparse_categorical_crossentropy` loss function, which is suited for multi-class classification problems.


In [35]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### **Training the Model with Data Augmentation**
We train the model using the data augmentation generator. Each epoch will see new variations of the images thanks to augmentation, helping improve the model's generalization ability.


In [36]:
model.fit(datagen.flow(train_images, train_labels, batch_size=64), epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
525/525 [==============================] - 22s 40ms/step - loss: 0.2202 - accuracy: 0.9377 - val_loss: 1.0395 - val_accuracy: 0.6602
Epoch 2/10
525/525 [==============================] - 21s 39ms/step - loss: 0.0871 - accuracy: 0.9754 - val_loss: 0.0640 - val_accuracy: 0.9794
Epoch 3/10
525/525 [==============================] - 22s 41ms/step - loss: 0.0670 - accuracy: 0.9795 - val_loss: 0.0582 - val_accuracy: 0.9801
Epoch 4/10
525/525 [==============================] - 22s 42ms/step - loss: 0.0549 - accuracy: 0.9844 - val_loss: 0.0418 - val_accuracy: 0.9863
Epoch 5/10
525/525 [==============================] - 22s 43ms/step - loss: 0.0516 - accuracy: 0.9844 - val_loss: 0.0315 - val_accuracy: 0.9901
Epoch 6/10
525/525 [==============================] - 22s 42ms/step - loss: 0.0447 - accuracy: 0.9868 - val_loss: 0.0398 - val_accuracy: 0.9871
Epoch 7/10
525/525 [==============================] - 22s 42ms/step - loss: 0.0438 - accuracy: 0.9873 - val_loss: 0.0717 - val_accuracy:

## Evaluating the Model
After training, we evaluate the model's performance on the validation set. This helps us understand the accuracy and generalization capabilities of the model.

### Making Predictions on Test Data
With the trained model, we make predictions on the test dataset. Each image is classified into one of the ten digit classes.

In [37]:
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

875/875 [==============================] - 6s 6ms/step


In [38]:
submission_df = pd.DataFrame({
    'ImageId': np.arange(1, len(predicted_classes) + 1),
    'Label': predicted_classes
})

### Creating the Submission File
We prepare the submission file by associating each test image with its predicted label. The submission file is formatted as required by Kaggle: each row contains an ImageId and the corresponding predicted Label.


In [39]:
submission_df = pd.DataFrame({
    'ImageId': np.arange(1, len(predicted_classes) + 1),
    'Label': predicted_classes
})

submission_df.to_csv('submission.csv', index=False)

# Conclusion

This code prepares the training data, applies real-time data augmentation during training, trains a convolutional neural network model, and saves the model for future use. Key steps include normalizing the images, splitting the data into training and validation sets, and applying data augmentation during training to improve model performance.
